In [1]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch.nn
import os
import mlflow
import mlflow.pytorch
import abc
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Iniciando MlFlow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
#Definição do Environment em que o Agente atuará
class PortfolioEnv(gym.Env):
    def __init__(self,arquivo_dados):
        """
        Inicializa o ambiente.
        :param dados_preco: DataFrame ou numpy array com preços dos ativos.
                           Exemplo: colunas ['Gold', 'IBOVESPA', 'Bitcoin']
        :param dados_observacoes: DataFrame ou numpy array com observação do modelo 
                           Exemplo: IPCA.
        """
        self.arquivo_dados = arquivo_dados
        dados = pd.read_csv(f'../data/{self.arquivo_dados}')
        self.dados_preco = dados.iloc[:, 1:4]  # Dataframe com preços dos ativos
        self.dados_observacao = dados.iloc[:, 4]  # Dataframe com Observações (IPCA)

        #Espaço de Ação e de Observação
        self.action_space = gym.spaces.Box(low=0.0, high=1.0, shape=(3,), dtype=np.float32)
        self.observation_space = gym.spaces.Box(low=0.0, high=np.inf, shape=(5,), dtype=np.float32)

        # Estado inicial do portfólio
        self._valor_inicial_portfolio = 100  # Valor inicial do portfólio
         
         #Total de Dias de um Episódio
        self.dias_em_um_episodio = 30

        #Episodio começa como não terminado
        self._episode_ended = False
    
    def get_estado(self, data: int) -> np.array:
        '''
        Obtém o estado atual do ambiente.

        :param data: Dia atual.
        :return: Estado atual.
        '''
        estado = np.concatenate([
            # alocacao,
            # [self._valor_portfolio],
            # self.dados_preco.iloc[self.dia],
            [self.dados_observacao.iloc[data]]
        ])
        return estado
    
    @abc.abstractmethod
    def calcula_recompensa(alocacao: torch.Tensor) -> torch.Tensor:
        pass

    #Função Chamada no Enviroment ao Fim de um Episódio, retorna o estado que será usado pelo o agente
    def reset(self):
        # definindo o dia inicial
        self.dia_inicial = np.random.randint(0, len(self.dados_preco) - self.dias_em_um_episodio)
        # na verdade, se passarmos 1 ano, terá que iniciar o np.random em 365
        self.dia = self.dia_inicial

        #Iniciado valor do portfólio para o valor inicial
        self._valor_portfolio = self._valor_inicial_portfolio

        self._episode_ended = False
        
        #Criando vetor de estado concatenado -> Necessário para entrada na Rede Neural
        estado = self.get_estado(self.dia_inicial)
        return estado
    
    #Função Chamada dentro de cada episódio, Recebe parametros da rede neualpara gerar uma amostra de uma distribuição normal
    def step(self,alocacao):
        recompensa = self.calcula_recompensa(alocacao)

        #Passa para o próximo dia
        self.dia += 1

        estado = self.get_estado(self.dia)

        #Alteração dentro da função step do ambiente
        if self.dia - self.dia_inicial == self.dias_em_um_episodio:
            self._episode_ended = True
        return estado, recompensa  # Retorna o estado atual e a recompensa ao invés de chamar reset()

In [4]:
class AmbienteAgenteConservador(PortfolioEnv):
    '''
    Implementa um ambiente de classe filha que implementa a função
    calcula_recompensa para um agente conservador.
    '''
    def calcula_recompensa(self, alocacao: torch.Tensor) -> torch.Tensor:
        '''
        Calcula a recompensa para um agente conservador.

        Args:
            alocacao: A alocação de ativos.
        
        Returns:   
            A recompensa.
        '''
        #Calcula a valorização do portfolio
        valor_portfolio_pre_aplicacao = self._valor_portfolio
        preco_usado = self.dados_preco.iloc[self.dia].values
        preco_dia_seguinte = self.dados_preco.iloc[self.dia + 1].values
        variacao_percentual = (preco_dia_seguinte-preco_usado)/preco_usado
        valores_aportados = torch.multiply(alocacao,self._valor_portfolio)
        valores_aportados_ajustados = torch.multiply(valores_aportados, torch.tensor(1 + variacao_percentual))
        self._valor_portfolio = torch.sum(valores_aportados_ajustados)

        #Calculo de Recompensa
        recompensa = torch.subtract(self._valor_portfolio,valor_portfolio_pre_aplicacao)

        #Ajustando Recompensa para aumentar grandeza de perdas
        if recompensa < 0:
            recompensa = recompensa*2

          
        return recompensa

In [5]:
class AmbienteAgenteArrojado(PortfolioEnv):
    '''
    Implementa um ambiente de classe filha que implementa a função
    calcula_recompensa para um agente conservador.
    '''
    def calcula_recompensa(self, alocacao: torch.Tensor) -> torch.Tensor:
        '''
        Calcula a recompensa para um agente conservador.

        Args:
            alocacao: A alocação de ativos.
        
        Returns:   
            A recompensa.
        '''
        #Calcula a valorização do portfolio
        valor_portfolio_pre_aplicacao = self._valor_portfolio
        preco_usado = self.dados_preco.iloc[self.dia].values
        preco_dia_seguinte = self.dados_preco.iloc[self.dia + 1].values
        variacao_percentual = (preco_dia_seguinte-preco_usado)/preco_usado
        valores_aportados = torch.multiply(alocacao,self._valor_portfolio)
        valores_aportados_ajustados = torch.multiply(valores_aportados, torch.tensor(1 + variacao_percentual))
        self._valor_portfolio = torch.sum(valores_aportados_ajustados)

        #Calculo de Recompensa
        recompensa = torch.subtract(self._valor_portfolio,valor_portfolio_pre_aplicacao)
         
        if recompensa < 0 :
            recompensa = 0.8 * recompensa
        elif recompensa > 0:
            recompensa = recompensa * 1.2
        return recompensa

In [6]:
#Definição de Rede Neural que servirá como Política
class PolicyNetwork(nn.Module):
    '''
    Observações:
    1. Necessitamos de um modo de gerar desvio padrão positivo e diferente de 0 
    2. Adicionar Saídas
    '''
    def __init__(self, state_dim, action_dim):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, 16)  # Camada oculta 1
        self.fc1_ativ = nn.ReLU()
        self.fc2 = nn.Linear(16, 16)         # Camada oculta 2
        self.fc2_ativ = nn.ReLU()
        self.fc_mu = nn.Linear(16, action_dim)  # Média (mu) dos pesos de portfólio
        self.output_ativ = nn.Sigmoid()
    
    def forward(self, x):
        x = self.fc1_ativ(self.fc1(x))  # Passa pela camada oculta 1
        x = self.fc2_ativ(self.fc2(x))      # Passa pela camada oculta 2
        mu = self.output_ativ(self.fc_mu(x))  # Média (mu), usando tanh para limitar a saída
        return mu

In [7]:
#Função para treinamento do Agente
def train(ambiente,politica,otimizador,num_episodios,):
    with mlflow.start_run():
        resultados_treino = pd.DataFrame(columns=["Episodio","Dia","Estado","Media","Desvio","Alocacao","Log_Prob","Recompensa"])
        for episodio in range(num_episodios):
            #Logar Parametros no MLFLow
            mlflow.log_params({"Número de Episódios":num_episodios,
                               "Learning Rate":otimizador.defaults['lr'],
                               "Arquivo Usado": ambiente.arquivo_dados})
            
            if isinstance(ambiente,AmbienteAgenteArrojado):
                mlflow.log_param("Perfil do Cliente","Arrojado")
            elif isinstance(ambiente,AmbienteAgenteConservador):
                mlflow.log_param("Perfil do Cliente","Conservador")
                
            estado = ambiente.reset() #No inicio do episodo pegamos o estado inicial do ambiente
            mlflow.log_param("Dimensão do Estado",len(estado))
            mlflow.log_param("Função de Ativação Output",politica.output_ativ)
            recompensas_do_episodio = [] #Array para guardar as recompensas de cada episodio
            log_probs = [] #Array para guardar as probabilidades logaritimicas usadas no calculo do Gradiente de Politica
            estados =[]
            alocacoes= []
            dias = []
            medias = []
            desvios = []
        
            while not ambiente._episode_ended:
                #Recebendo Estado
                estado_tensor = torch.tensor(estado, dtype=torch.float32) #Transformarmos em tensor -> Requisitado pelo Pytorch
            # media,desvio_padrao = politica(estado_tensor) #Geramos os parametros para criação da distribuição da ação
                media = politica(estado_tensor)
                medias.append(media)   
                desvio_padrao = torch.tensor([0.1,0.1,0.1])
                desvios.append(desvio_padrao)
            # desvios.append(desvio_padrao)

                #Adicionando dia
                dias.append(ambiente.dia)

                #Gerando Alocação
                # Função para amostrar os pesos do portfólio
                def sample_portfolio_weights(mu, sigma):
                    dist = torch.distributions.Normal(mu, sigma)  # Cria a distribuição normal
                    action = dist.rsample()  # Amostragem reparametrizada
                    log_prob = dist.log_prob(action).sum(dim=-1)  # Log-probabilidade da ação
                    action = torch.clamp(action,0, 1)  # Limita os valores entre 0 e 1
                    action = action/torch.sum(action)
                    return action, log_prob
                
                # Amostra os pesos do portfólio
                alocacao, log_prob = sample_portfolio_weights(media,desvio_padrao)
                

                #Guardando Estado e Alocação
                estados.append(estado)
                alocacoes.append(alocacao)

                #Calculando log-probabilidade da ação
                log_probs.append(log_prob)

                #Executa a alocação e recebe a recompensa
                retorno = ambiente.step(alocacao)
                estado = retorno[0] #Atualiza o estado para o novo estado devolvido pelo ambiente e pega a recompensa da ação anterior
                recompensa = retorno[1]
                recompensas_do_episodio.append(recompensa)
        
            #Calculo de Perda -> Gradiente de Politica:
            perda = -torch.sum(torch.stack(log_probs) * torch.stack(recompensas_do_episodio))
            otimizador.zero_grad()
            perda.backward()
            otimizador.step()
            
            # Adicionar resultados em um dataframe para análise do treinamento
            for dia, estado, media, desvio, alocacao, log_prob, recompensa in zip(dias, estados, medias, desvios, alocacoes, log_probs, recompensas_do_episodio):
                resultados_treino.loc[len(resultados_treino)]= {
                    "Episodio": episodio,
                    "Dia": dia,
                    "Estado": estado,
                    "Media": media.detach().numpy(),
                    "Desvio": desvio.detach().numpy(),
                    "Alocacao": alocacao.detach().numpy(),
                    "Log_Prob": log_prob.detach().numpy(),
                    "Recompensa": recompensa.detach().numpy(),
                }
        
        #Salvando Treino 
        def salvartreino():
            media_final = medias[-1].detach().numpy()
            for i, valor in enumerate(media_final):
                mlflow.log_metric(f"Média Final_{i}", valor)

            # Desvio Final
            desvio_final = desvios[-1].detach().numpy()
            for i, valor in enumerate(desvio_final):
                mlflow.log_metric(f"Desvio Final_{i}", valor)

            # Alocação Final
            alocacao_final = alocacoes[-1].detach().numpy()
            for i, valor in enumerate(alocacao_final):
                mlflow.log_metric(f"Alocação Final_{i}", valor)

            #Recompensa Média por Episódio
            mlflow.log_metric("Recompensa Média por Episódio", resultados_treino.groupby('Episodio')['Recompensa'].sum().mean())
            mlflow.log_metric("Dias por Episódio", ambiente.dias_em_um_episodio)
            max_n = 0
            for i in os.listdir('../data/resultados_treinos/v1.1'):
                n = i.split('_')[1]
                n = int(n.split('.')[0])
                if int(n) > max_n:
                    max_n = n 
            max_n = max_n +1
            resultados_treino.to_csv(f"../data/resultados_treinos/v1.1/treino_{max_n}.csv", index=False)
             # Logar o arquivo no MLflow
            mlflow.log_artifact(f"../data/resultados_treinos/v1.1/treino_{max_n}.csv")
            mlflow.pytorch.log_model(pytorch_model=politica,artifact_path='Magnum')
        salvartreino()
       
        return resultados_treino

In [8]:
#Treino Conservador
ambiente = AmbienteAgenteConservador('merged_v1.csv') #Criando Ambiente
politica = PolicyNetwork(1, 3) #Criando Politica Estocástica
otimizador = optim.Adam(politica.parameters(),lr=0.01) #Cria otimizador associado aos parâmetros da rede a ser atualizada
resultados_treino = train(ambiente,politica,otimizador,num_episodios=1000,fator_desconto_recompensa=0.99)

,Episodio,Dia,Estado,Media,Desvio,Alocacao,Log_Prob,Recompensa
0,0,1006,[0.25],"[0.53481615, 0.445804, 0.5828185]","[0.1, 0.1, 0.1]","[0.35728905, 0.30900794, 0.333703]",3.8602529,-1.1001859792784217
1,0,1007,[0.25],"[0.53481615, 0.445804, 0.5828185]","[0.1, 0.1, 0.1]","[0.36753818, 0.27784604, 0.35461575]",3.8465328,-0.5469297327019262
2,0,1008,[0.25],"[0.53481615, 0.445804, 0.5828185]","[0.1, 0.1, 0.1]","[0.34265584, 0.24606055, 0.4112836]",3.7973676,0.1297743404717835
3,0,1009,[0.25],"[0.53481615, 0.445804, 0.5828185]","[0.1, 0.1, 0.1]","[0.34130248, 0.3369496, 0.32174796]",3.3464823,1.8515426842391207
4,0,1010,[0.25],"[0.53481615, 0.445804, 0.5828185]","[0.1, 0.1, 0.1]","[0.33856997, 0.30925545, 0.35217458]",4.0254183,-1.9091334048706017


In [13]:
def plot_df(df,nomes_ativos):
    # descarte pegue só a primeira linha de cada valor diferente de episódio com np.unique
    df = df.loc[df.groupby('Episodio').cumcount() == 0]

    # transforme a coluna alocação em um vetor
    df['Alocacao'] = df['Alocacao'].apply(lambda x: np.array([float(i) for i in x[1:-1].split()]))

    df['Alocacao_Gold'] = df['Alocacao'].apply(lambda x: x[0])
    df['Alocacao_Bitcoin'] = df['Alocacao'].apply(lambda x: x[1])
    df['Alocacao_Ibovespa'] = df['Alocacao'].apply(lambda x: x[2])

    # Dados de exemplo
    x = df['Episodio']
    y1 = df['Alocacao_Gold']  # Primeira variável no eixo y
    y2 = df['Alocacao_Bitcoin']  # Segunda variável no eixo y
    y3 = df['Alocacao_Ibovespa']  # Terceira variável no eixo y
    y4 = df['Recompensa']

    # Criação do gráfico
    fig = go.Figure()

    # Primeira série de dados (para y1)
    fig.add_trace(go.Scatter(
        x=x, 
        y=y1, 
        mode='lines', 
        name=nomes_ativos[0],
        line=dict(width=1),  # Diminuindo a espessura da linha
        marker=dict(size=1)  # Diminuindo o tamanho dos marcadores
    ))

    # Segunda série de dados (para y2)
    fig.add_trace(go.Scatter(
        x=x, 
        y=y2, 
        mode='lines', 
        name=nomes_ativos[1],
        line=dict(width=1),  # Diminuindo a espessura da linha
        marker=dict(size=1)  # Diminuindo o tamanho dos marcadores
    ))

    # Terceira série de dados (para y3)
    fig.add_trace(go.Scatter(
        x=x, 
        y=y3, 
        mode='lines', 
        name=nomes_ativos[2],
        line=dict(width=1),  # Diminuindo a espessura da linha
        marker=dict(size=1)  # Diminuindo o tamanho dos marcadores
    ))

    # Quarta série de dados (para y4)
    fig.add_trace(go.Scatter(
        x=x, 
        y=y4, 
        mode='lines', 
        name='Recompensa',
        line=dict(width=1),  # Diminuindo a espessura da linha
        marker=dict(size=1)  # Diminuindo o tamanho dos marcadores
    ))

    # Título e labels dos eixos
    fig.update_layout(
        title="Gráfico de Alocação de Ativos ao Longo do Tempo",
        xaxis_title="Episódio",
        yaxis_title="Porcentagem de Alocação",
    )
    # Mostrar o gráfico
    fig.show()
df = pd.read_csv('../data/resultados_treinos/v1.1/treino_72.csv')
plot_df(df,['Ouro','Bitcoin','Ibovespa'])

In [18]:
#Treino Arrojado
ambiente = AmbienteAgenteArrojado('merged_v1.csv') #Criando Ambiente
politica = PolicyNetwork(1, 3) #Criando Politica Estocástica
otimizador = optim.Adam(politica.parameters(),lr=0.1) #Cria otimizador associado aos parâmetros da rede a ser atualizada
resultados_treino = train(ambiente,politica,otimizador,num_episodios=1000,fator_desconto_recompensa=0.99)